# Colab Demo

In [ ]:
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


## Dependencies and Imports

In [ ]:
#@title Install dependencies

import os
import yaml
import torch
from torch import package

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

In [ ]:
print('hello')

hello


In [ ]:
# see avaiable languages
available_languages = list(model_conf.get('languages'))
print(f'Available languages {available_languages}')

# and punctuation marks
available_punct = list(model_conf.get('punct'))
print(f'Available punctuation marks {available_punct}')

Available languages ['en', 'de', 'ru', 'es']
Available punctuation marks ['.', ',', '-', '!', '?', '—']


## With language label

In [ ]:
model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))

if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)

imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")
example_texts = model.examples

def apply_te(text, lan='ru'):
    return model.enhance_text(text, lan)

#### Input from the keyboard

In [ ]:
dev_video_text = open('youtube_scr.txt','r').read()

In [ ]:
dev_video_text = dev_video_text.split('\n')
dev_video_text = [text for i,text in enumerate(dev_video_text) if text and not text[0].isdigit()]

In [ ]:
def chunk_text_by_words(text, max_words=5):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(current_chunk) < max_words:
            current_chunk.append(word)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [ ]:
import re
dev_video_script = ""
for text in dev_video_text:
    text = text.lower()
    text = re.sub(r'\[.*\]','',text)
    text = re.sub(r'«','',text)
    text = re.sub(r'»','',text)
    text = re.sub(r'-','',text)
    text = re.sub(r'“','',text)
    text = re.sub(r'\.','',text)
    print(text)
    s = apply_te(text, lan='ru')
    dev_video_script+=s
    dev_video_script+=" "

лёня это когда сходил в душ последний раз на первом курсе например там доказывали что 0 меньше чем один мы
некий эталон того кем ты хочешь стать это теплом от а давай немножко повысим градус да вот мы всетаки чем ты занимаешься все как бы зависит от всех
факторов
всем привет меня зовут женя соколов я работаю на факультете компьютерных наук в высшей школе экономики и мы решили
новокаин запустить серию подкастов про то как изучать компьютерные науки что
там есть как там устроена наука и так далее и первый наш выпуск будет про математику попробую объяснить почему во
первых я довольно много выступаю на разных мероприятиях часто выступаю перед совершенно неподготовленной аудитории
меня разумеется спрашивают как в катится в компьютерные науки в программировании
в аналитику в да это сайнс я чтото отвечаю советую какието курсы еще меня спрашивают а вот что нужно знать перед
тем как начинать то изучать но я говорю что конечно математику и тут встает уже куда более сложный вопрос а почему
матема

In [ ]:
with open('script5.txt','w') as f:
    f.write(dev_video_script)

In [ ]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c468125789b1802c33b5d6f1903741898c0376fec75a993e72d8ac5c145d4b88
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [ ]:
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)


def pymorphy2_example():
    import pymorphy2
    pymorphy2_311_hotfix()
    morph = pymorphy2.MorphAnalyzer()
    parse_result = morph.parse('хочу')
    print(parse_result[0].normal_form)  # >>> хотеть
pymorphy2_example()

хотеть


In [ ]:
import gensim
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import pymorphy2
from tqdm import tqdm
from inspect import getfullargspec
pymorphy2_311_hotfix()
morph = pymorphy2.MorphAnalyzer()


necessary_part = {"NOUN", "ADJF", "ADJS", "VERB", "INFN", "PRTF", "PRTS", "GRND"}
with open('script1.txt', 'r', encoding='utf8') as f:
    text = f.read().split('\n')
    sentences = []

    # Normalization
    for line in text:
        sentences.append(text_to_word_sequence(line))

    for i in tqdm(range(len(sentences))):
        sentence = []
        for el in sentences[i]:
            p = morph.parse(el)[0]
            if p.tag.POS in necessary_part:
                sentence.append(p.normal_form)
        sentences[i] = sentence
    sentences = [x for x in sentences if x]
    model_name = 'my.model'


    model = gensim.models.FastText(sentences,  window=3, min_count=2, sg=1)

    model.init_sims(replace=True)

    model.save(model_name)

    model = gensim.models.FastText.load(model_name)
    model.init_sims(replace=True)

    words = dev_video_script.split()

    for index, word in enumerate(words):
        if word in model.wv.index_to_key:
            if word not in model.wv.most_similar(positive=[word], topn=20):
                print(word)


100%|██████████| 1/1 [00:01<00:00,  1.18s/it]
<ipython-input-30-d5465486c9b3>:32: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)
<ipython-input-30-d5465486c9b3>:37: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


искусственный
интеллект
некоторый
вид
тема
есть
разобраться
разработчик
руководитель
привет
свой
карьера
такой
учиться
разработка
заниматься
время
путь
разрабатывать
работать
тот
команда
руководитель
есть
группа
мир
есть
сайт
весь
этот
весь
свой
тот
конкурировать
сделать
рынок
свой
рынок
часть
есть
куча
есть
рынок
есть
ценность
развиваться
ответ
этот
имплементировать
сделать
есть
есть
есть
приложение
пример
сделать
этот
рынок
разработка
есть
разделение
делать
сделать
сделать
есть
переходить
деление
кроссплатформенный
разработчик
нативный
разработчик
каста
опыт
есть
есть
ребята
есть
разрабатывать
который
экосистема
есть
некоторый
есть
один
фактор
сказать
есть
написать
один
универсальный
который
учёт
сделать
раз
договориться
конкурировать
есть
развивать
свой
продукт
есть
есть
есть
есть
один
быть
быть
есть
разработка
разработка
разработка
старый
погружаться
база
дело
есть
поговорить
знать
область
есть
знать
знать
интернет
есть
иметь
выложить
приложение
банк
раз
есть
главное
есть
есть
есть

In [ ]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.9 MB/s eta 0:00:00


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')
text = "Привет, как делишки?"
corrected_text = " ".join([spell.correction(word) for word in text.split()])
print(corrected_text)  # Привет, как дела?

привет как детишки


In [ ]:
pip install language-tool-python

In [ ]:
import language_tool_python

tool = language_tool_python.LanguageTool('ru')

In [ ]:
dev_video_script = open('/content/script2.txt','r').read()

In [ ]:
dev_video_script

"На какую работу нужно пойти, чтобы вот это делать, потому что у тебя за деньги платили они просто оценки ставили и вряд. ли сейчас можно на какуюто найти такую болееменее интересную содержательную работу, которая бы тебя значит там не Погружал бы просто с головой В этот наш ужасный мир пожалуйста, Давайте спарсинца. Как бы, не включая мозг, придумал какуюто схему очень сложную, но типа она мне казалась разумной, а студент он э в. принципе хочет а максимизировать, а с своей результаты и минимизировать усилия. И это я так преподаю! Всем привет! Это очередной выпуск подкаста уютный фокаэнчик, где мы говорим о компьютерных науках. Мы на факультете. компьютерных наук в вышке довольно неплохо умеем учить специалистов разработчиков, Дата сайнтистов аналитиков. И так далее, но сегодня вот как раньше английский стал Чемто, что должны знать. Все, а не только профессионалы. Сегодня компьютерные науки становятся чемто, в чем должны хоть немного разбираться все. И вот сегодня мы как раз поговорим 

In [ ]:
matches = tool.check(dev_video_script)
corrected_dev = language_tool_python.utils.correct(dev_video_script, matches)

In [ ]:
with open('script15.txt','w') as f:
    f.write(dev_video_script)

In [ ]:
import json


scripts = {}

for i in range(1, 16):
    filename = f'script{i}.txt'
    with open(filename, 'r') as file:
        content = file.read()
        scripts[str(i)] = content

with open('scripts.json', 'w') as json_file:
    json.dump(scripts, json_file, ensure_ascii=False, indent=4)

print("Скрипты успешно объединены в файл scripts.json")


Скрипты успешно объединены в файл scripts.json
